# AI_A6
## Kasra Ghorbani
## 810101489


## Task :
Create a classic snake game and train the 2 snakes using Qlearning so that they can play against each other.
## States :
Since evaluating all the possible states which includes taking into account all the available houses alongside all possible snake and food locations as well as other factors is simply unachievable we reduced and simplified the observation space. How?
We only observe the 9 other squares other than the head of the snake. The way this works is that we create a grid around the head of the snake which in this case has a surface of 9 and check the different ways that it can be occupied including being occupied by the opponent snake, the snake's own body and if its out of bounds. and we return a tuple which contains 0s and 1s for when the corresponding cell around the head is occupied or free. The other thing we take into consideration is the direction of the apple relative to the snake so that if the apple is 6 blocks to the right of the snake and 2 blocks above we prioritize moving to the right. And with these our state is defined.

In [ ]:
def create_state(self, snack, other_snake):
    neighbor = self.get_neighbor(3, other_snake)
    snack_side_from_snake = self.calc_snack_side_from_snake(snack)
    return neighbor, snack_side_from_snake
    
def get_neighbor(self, size, other_snake): #if cell is free append 1 if occupied append 0
    #create a grid based on given size
    distance = (size - 1) // 2
    tmp = np.array(range(distance + 1))
    tmp = np.union1d(tmp, -tmp)
        
    output = []
    for i in tmp + self.head.pos[0]: #grid is relative to head location
        for j in tmp + self.head.pos[1]:
            if i < 1 or i >= ROWS - 1 or j < 1 or j >= ROWS - 1:
                output.append(0)
            elif (i, j) in list(map(lambda z: z.pos, self.body)): #if cell occupied by own body
                output.append(0)
            #if cell occupied by other snake
            elif (i, j) in list(map(lambda z: z.pos, other_snake.body)) or (i, j) == other_snake.head.pos:
                output.append(0)
            else:
                output.append(1)
    return tuple(output)
def calc_snack_side_from_snake(self, snack): #check if the snack is LRUD relative to the snakes head

    if abs(snack.pos[0] - self.head.pos[0]) >= abs(snack.pos[1] - self.head.pos[1]):
        if snack.pos[0] < self.head.pos[0]:
            return 0 #L
        if snack.pos[0] > self.head.pos[0]:
            return 1 #R
    else:
        if snack.pos[1] < self.head.pos[1]:
            return 2 #U
        if snack.pos[1] > self.head.pos[1]:
            return 3 #D
    return -1

## The optimal Policy :
We simply choose the highest qreward from the qtable if the snakes decides not to explore : 

In [ ]:
def get_optimal_policy(self, state): #look into qtable and return the best action based on knowledge (optimal policy)
    return np.argmax(self.q_table[state])
    
def make_action(self, state):
    chance = random.random()
    if chance < self.epsilon: #if rand num lower than exploration rate choose rand action
        action = random.randint(0, 3)
    else:
        action = self.get_optimal_policy(state) #if not play by policy
    return action